In [ ]:
import yaml
import cv2
import numpy as np
import PIL.Image
import time
from pynq.lib.video import *
from pynq.lib import Pmod_PWM
from pynq.lib.arduino import Arduino_IO
from pynq.overlays.base import BaseOverlay

In [ ]:
## Line detection ##

def line_detection(roi_rate, frame):
    
    # Read input img size
    img_h = frame.shape[0]
    img_w = frame.shape[1]
    
    # Take lower part of the frame
    roi = frame[int(img_h*roi_rate):, :]
    
    # Canny detect edges
    edges = cv2.Canny(roi, 39, 130)

    # Hough Line detect lines, output lines (x1, y1, x2, y2)
    lines = cv2.HoughLinesP(
        edges,
        rho=1.0, # The resolution of the parameter r in pixels. We use 1 pixel.
        theta=np.pi/180, # The resolution of the parameter θ in radians. We use 1 degree (CV_PI/180)
        threshold=8, # The minimum number of intersections to "*detect*" a line
        minLineLength=10, # The minimum number of points that can form a line. Lines with less than this number of points are disregarded.
        maxLineGap=9 # The maximum gap between two points to be considered in the same line.
    )
            
    return lines

In [ ]:
## Find position offset from middle of two lines ##

def offset_from_middle(roi_rate, lines, overlay):

    if lines is None:
        return 144, overlay
    
    else:
        # Read input img size
        img_h = overlay.shape[0]
        img_w = overlay.shape[1]
        
        # Find the two lines that farthest from the middle 
        line_count = -1
        left_line_num = -1
        right_line_num = -1
        
        left_line_x1 = img_w/2
        right_line_x2 = img_w/2

        for line in lines:
            line_count = line_count + 1
            for x1, y1, x2, y2 in line:
                
                # Exclude lines that are less than 30 degree to the horizontal
                # tan(30) = 0.58
                if (abs(y2-y1) > 0.58 * abs(x2-x1)):
                    
                    # Find left line
                    if (x1 < img_w/2 and x1 < left_line_x1):
                        left_line_x1 = x1
                        left_line_num = line_count

                    # Find right line
                    if (x2 > img_w/2 and x2 > right_line_x2):
                        right_line_x2 = x2
                        right_line_num = line_count
                        
        # Create new empty frame for drawing
        dot_img = np.zeros((img_h, img_w, 1), dtype=np.uint8)
        
        # Draw frame center line
        cv2.line(dot_img, (int(img_w/2), 0), (int(img_w/2), img_h), 100, 1)
        
        # Draw left line
        for x1, y1, x2, y2 in lines[left_line_num]:
            #y=kx+b
            left_b = x1
            #draw dot
            cv2.circle(dot_img, (x1, y1+int(img_h*roi_rate)), 3, 233, 0)
            cv2.circle(dot_img, (x2, y2+int(img_h*roi_rate)), 3, 233, -1)
            #draw line
            cv2.line(dot_img, (x1, y1+int(img_h*roi_rate)), (x2, y2+int(img_h*roi_rate)), 233, 1)

        # Draw right line
        for x1, y1, x2, y2 in lines[right_line_num]:
            #y=kx+b
            right_b = x2
            #draw dot
            cv2.circle(dot_img, (x1, y1+int(img_h*roi_rate)), 3, 233, 0)
            cv2.circle(dot_img, (x2, y2+int(img_h*roi_rate)), 3, 233, -1)
            #draw line
            cv2.line(dot_img, (x1, y1+int(img_h*roi_rate)), (x2, y2+int(img_h*roi_rate)), 233, 1)

        # Draw middle point of two lines
        mid_dot = int((left_b + right_b)/2)
        cv2.circle(dot_img, (mid_dot, img_h), 3, 233, -1)
        
        # Combine drawings with input frame
        overlay = cv2.addWeighted(overlay, 1.0, dot_img, 1.0, 0.0)
        
        return mid_dot, overlay

In [ ]:
## Car detection ##

def car_detection(frame):

    # Read input img size
    img_h = frame.shape[0]
    img_w = frame.shape[1]
   
    # Pass frame to our car classifier
    cars = car_classifier.detectMultiScale(frame)
    
    # draw lines
    zero_img = np.zeros((img_h, img_w, 1), dtype=np.uint8)

    # Extract bounding boxes for any bodies identified
    for (x,y,w,h) in cars:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (233), 1)
    
    #overlay = cv2.addWeighted(frame, 1, zero_img, 1.0, 0.0)
    
    return cars ,frame

In [ ]:
## Format frame for HDMI output ##

def format_frame(hdmi_out, frame):
    
    # Read input frame size
    img_h = frame.shape[0]
    img_w = frame.shape[1]
    
    # Create new video frame
    outframe = hdmi_out.newframe()
    
    # Fill new video frame with zero
    zero_img = np.zeros((480, 640), dtype=np.uint8)
    outframe[0:480,0:640] = zero_img[0:480,0:640]

    # Transfer input frame to the video frame
    # For Gray imge
    outframe[0:img_h,0:img_w] = frame[0:img_h,0:img_w]
    
    return outframe

In [ ]:
## Robot stop ##

def robot_stop():
    enable_L.stop()
    forward_L.write(0)
    backward_L.write(0)

    enable_R.stop()
    forward_R.write(0)
    backward_R.write(0)

In [ ]:
## Robot perportional control ##

def robot_run(left_percent, right_percent):
    
    left_duty = int(53 + (25) * left_percent - (30) * right_percent)
    right_duty = int(53 + (25) * right_percent - (30) * left_percent)

    
    enable_L.generate(10,left_duty)
    forward_L.write(1)
    backward_L.write(0)

    enable_R.generate(10,right_duty)
    forward_R.write(1)
    backward_R.write(0)

In [ ]:
## Initialize HDMI I/O ##

# Load the overlay
base = BaseOverlay("base.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

# Configure HDMI input to gray scale ( (0.3 * R) + (0.59 * G) + (0.11 * B) )
hdmi_in.configure(PIXEL_GRAY)
hdmi_in.start()

# Configure Output resolution (w, h, bit per pixek)
hdmi_out_mode = VideoMode(640,480,8)
hdmi_out.configure(hdmi_out_mode, PIXEL_GRAY)
hdmi_out.start()

In [ ]:
## Read camera calibration parameter ##

fname = "calibration_parameter.yaml"
with open(fname) as file:
    data = yaml.load(file,Loader=yaml.Loader)
    
mtx = np.array([ [ data[0] , 0, data[1] ] , [ 0, data[2], data[3] ] , [0, 0, 1] ])
dist = np.array([ [ data[4], data[5], data[6], data[7], data[8] ] ])

In [ ]:
## Initialize on borad LED and switch ##

led1 = base.leds[1] #Corresponds to LED LD1
led2 = base.leds[2] #Corresponds to LED LD2
led4 = base.rgbleds[4] #Corresponds to LED LD3
led5 = base.rgbleds[5] #Corresponds to LED LD4

sw0 = base.switches[0] #Corresponds to SW0

In [ ]:
## Initialize Arduino pins for motor control ##

enable_L  = Pmod_PWM(base.PMODA,1)
forward_L = Arduino_IO(base.ARDUINO, 7, 'out')
backward_L = Arduino_IO(base.ARDUINO, 8, 'out')

enable_R = Pmod_PWM(base.PMODB,1)
forward_R = Arduino_IO(base.ARDUINO, 11, 'out')
backward_R = Arduino_IO(base.ARDUINO, 9, 'out')

In [ ]:
## Main ##

# ROI for line dection set to the lower 1/4 of the frame
roi_rate = 3/4

# Load car harr classifier
car_classifier = cv2.CascadeClassifier('haar_toy_car.xml')

while(True):
    # Read frame from HDMI input
    frame = hdmi_in.readframe()
    
    # Resize frame (half of the 720p input)
    img = cv2.resize(frame, (640,360))
    
    # Undistort input frame
    h,  w = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
    
    # Crop the frame based on undistortion roi
    x,y,w,h = roi
    #dst = dst[y:y+h, x:x+w]
    
    # Crop the frame to align visual center of the frame
    dst = dst[y:y+h, x+120:x+w-150]

    # Detect car
    cars, cars_overlay = car_detection(dst)
    
    # Stop the robot when car detected, show status with LED
    if(len(cars) > 0):
        robot_stop()
        led5.on(4)
        
    if(len(cars) == 0):
        led5.on(3)
    
    # Detect line
    lines = line_detection(roi_rate, dst)
    
    # Find lines offset to the middle of the frame
    mid_dot, offset_overlay = offset_from_middle(roi_rate, lines, dst)
    
    # Follow the lines with perportional control
    mid_line = offset_overlay.shape[1]/2
    
    right_percent = 0
    left_percent = 0
    
    if(mid_dot < mid_line):
        right_percent = abs(mid_dot - mid_line)/69
        if (right_percent >= 1):
            right_percent = 1
        led1.on()
        led2.off()
        #print("right", mid_line)
        
    if(mid_dot > mid_line):
        left_percent = abs(mid_dot - mid_line)/69
        if (left_percent >= 1):
            left_percent = 1
        led2.on()
        led1.off()
        #print("left", mid_line)
    
    # Real Switch on/off to generate PWM signal
    if ((sw0.read() == True) and (len(cars) == 0)):
        led4.write(3)
        robot_run(left_percent, right_percent)
        
    if (sw0.read() == False):
        led4.write(4)
        robot_stop()
    
    # Conbine vehicle and line dection output frame
    overlay = cv2.addWeighted(offset_overlay, 0.5, cars_overlay, 0.5, 0.0)

    # Output to HDMI
    outframe = format_frame(hdmi_out, overlay)
    hdmi_out.writeframe(outframe)   

In [ ]:
led1.off()
led2.off()
led4.off()
led5.off()

In [ ]:
hdmi_out.stop()
hdmi_in.stop()
del hdmi_in, hdmi_out